In [1]:
!pip install -U pymarc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 KB 3.2 MB/s eta 0:00:00 MB/s eta 0:00:01


In [50]:
%time
from pymarc import MARCReader
import zipfile
import sqlite3

# open the zip file of the marc data
with zipfile.ZipFile('2023-01-24_newsdex-full-marc-export.zip') as marc_data_file:
    # should only contain one file ...
    print(marc_data_file.filelist[0].filename)
    # get a file handle for the marc data file
    with marc_data_file.open(marc_data_file.filelist[0]) as fh:
        reader = MARCReader(fh, to_unicode=True, force_utf8=True)
        for i, record in enumerate(reader):
            print(i, record['001'].value(), record.title(), [subject.value() for subject in record.subjects()])
            if i>=19:
                break

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.68 µs
2023-01-24_newsdex-full-export.out
0 NEWS-93 Let The Sunshine In []
1 NEWS-120 Bengals Romp But Browns Ruin Title Bid ['Cincinnati Bengals']
2 NEWS-121 Chance Of White Christmas: 100% ['Christmas', 'Snow']
3 NEWS-122 Pilot Killed At Lunken ['Ott, Robert Jeffrey', 'Airplane Accidents', 'Lunken Airport']
4 NEWS-123 Dr. Aring Is Man Of Many Gifts ['Aring, Charles D. (Dr.)']
5 NEWS-125 Schools Get Aid With Problems ['Sage Analytics International', 'Woodward High School', 'Cincinnati Public Schools, Problem Identifying Programs']
6 NEWS-126 Radio-Tower Proposal Gets Hearing In Court ['Hoker Broadcasting Co.', 'Wofx-Fm, Radio Tower', 'College Hill']
7 NEWS-127 Evanston Man Jailed On Murder Charges ['Maupin, Christopher L.', 'Sharma, Abhay D.', 'Murders']
8 NEWS-128 Recycling-Prize Winners Invited To New Drawing ['Recycling (Cincinnati)', 'Browning-Ferris Industries Inc.']
9 NEWS-133 America's Unending Revolution ['Spiegel, S. Ar

In [35]:
[field.tag for field in record.fields]

['001',
 '008',
 '245',
 '260',
 '650',
 '650',
 '650',
 '773',
 '907',
 '998',
 '944',
 '941',
 '959']

In [48]:
# record['773'].subfields_as_dict()
[field.value() for field in record.get_fields('650')]

['Sizer, Tom', 'Sizer, Jeff', "Pkg's Incorporated"]

In [37]:
# 773 - Host Item Entry (R)
# $a - Main entry heading (NR)
# $g - Related parts (R)
(record['773']['a'], record['773']['g'])

('Cincinnati Enquirer', '12/18/1989 D:1')

In [72]:
# record['260'].value()
# publication info
# pub_year
record['260']['c'].format()

'1989'

In [38]:
record['773']['a'].format()

'Cincinnati Enquirer'

In [59]:
[(field.subfields) for field in record.get_fields('773')]

[['a', 'Cincinnati Enquirer', 'g', '12/18/1989 D:1']]

In [64]:
fields = record.get_fields('650')

In [68]:
fields[2].subfields_as_dict()

defaultdict(list, {'a': ["Pkg's Incorporated"]})

In [16]:
# 260 $c - Date of publication, distribution, etc. (R)
record['260']['c']

'1'

In [77]:
len(record.get_fields('650'))

2

In [82]:
for i, value in enumerate([field.value() for field in record.get_fields('650')]):
    print(record['001'].value(), i, value)

NEWS-241 0 Recycling (Wyoming)
NEWS-241 1 Recycling


In [ ]:
# with open('2023-01-24_newsdex-full-export.out', 'rb') as fh:
#     reader = MARCReader(fh)
#     for i, record in enumerate(reader):
#         print(i, record.title(), [subject.value() for subject in record.subjects()])
#         if i >= 6:
#             break

In [39]:
# note this doesn't work for repeating fields (like the 650)
for tag in [field.tag for field in record.fields]:
    print(tag, record[tag].value())

001 NEWS-126
008                                         
245 Radio-Tower Proposal Gets Hearing In Court
260 1989
650 Hoker Broadcasting Co.
650 Hoker Broadcasting Co.
650 Hoker Broadcasting Co.
773 Cincinnati Enquirer 12/18/1989 A:10
907 .b10030098 10-03-12 08-31-12
998 2ma 07-23-04 m 9 - eng 0
944 7
941 20040723 0 BATCH 0
959 Newspaper Article MAIN MICROFILM MAGSTACKS MICROFORM N Y $0.00


In [23]:
print([f.value() for f in record.get_fields('650')])

['Hoker Broadcasting Co.', 'Wofx-Fm, Radio Tower', 'College Hill']


In [42]:
repeat_tags = []
for tag in [field.tag for field in record.fields]:
    # print(tag, len(record.get_fields(tag)), end='')
    if len(record.get_fields(tag)) == 1:
        # print(tag, record[tag].value())
        try:
            print(tag, dict(record[tag].subfields_as_dict()))
        except:
            print(tag, record[tag].value())
    else:
        repeat_tags.append(tag)
        
    # print(
    #     # tag, 
    #     # record[tag].value())
    #     [f.value() for f in record.get_fields(tag)]
    # )

001 NEWS-144
008                                         
245 {'a': ['Food Parcel Proves Popular']}
260 {'c': ['1989']}
773 {'a': ['Cincinnati Enquirer'], 'g': ['12/18/1989 D:1']}
907 {'a': ['.b10030220'], 'b': ['10-03-12'], 'c': ['08-31-12']}
998 {'a': ['2ma'], 'b': ['07-23-04'], 'c': ['m'], 'd': ['9'], 'e': ['-'], 'f': ['eng'], 'h': ['0']}
944 {'a': ['20']}
941 {'a': ['20040723'], 'b': ['0'], 'c': ['BATCH'], 'd': ['0']}
959 {'a': ['Newspaper Article'], 'm': ['MAIN'], 't': ['MICROFILM'], 'l': ['MAGSTACKS'], 'x': ['MICROFORM'], 'r': ['N'], 's': ['Y'], 'p': ['$0.00']}


In [44]:
field = record['907']

In [45]:
# field.is_subject_field()
print(dict(field.subfields_as_dict()))

{'a': ['.b10030098'], 'b': ['10-03-12'], 'c': ['08-31-12']}


In [43]:
len(record.get_fields('650'))

3

In [46]:
for sub in record.subjects():
    print(sub.value())

Hoker Broadcasting Co.
Wofx-Fm, Radio Tower
College Hill


In [47]:
print([field.tag for field in record.fields])

['001', '008', '245', '260', '650', '650', '650', '773', '907', '998', '944', '941', '959']


In [64]:
field = record['650'].format_field()

In [70]:
record['001'].value()

'NEWS-126'

In [62]:
print([(field.tag, field.value(), ) for field in record.fields])

AttributeError: 'Field' object has no attribute 'subfields'

In [64]:
print([(field.tag, field.value()) for field in record.fields if field.tag >= '900'])

[('907', '.b10030098 10-03-12 08-31-12'), ('998', '2ma 07-23-04 m 9 - eng 0'), ('944', '7'), ('941', '20040723 0 BATCH 0'), ('959', 'Newspaper Article MAIN MICROFILM MAGSTACKS MICROFORM N Y $0.00')]


In [68]:
print(record.get_fields())

[<pymarc.field.Field object at 0x7fe94162a3b0>, <pymarc.field.Field object at 0x7fe94162af20>, <pymarc.field.Field object at 0x7fe94162a260>, <pymarc.field.Field object at 0x7fe94162a800>, <pymarc.field.Field object at 0x7fe94162ab90>, <pymarc.field.Field object at 0x7fe941854310>, <pymarc.field.Field object at 0x7fe941854670>, <pymarc.field.Field object at 0x7fe941824f40>, <pymarc.field.Field object at 0x7fe941827550>, <pymarc.field.Field object at 0x7fe941743580>, <pymarc.field.Field object at 0x7fe941789060>, <pymarc.field.Field object at 0x7fe942429330>, <pymarc.field.Field object at 0x7fe941827ac0>]


In [69]:
record.as_json()

'{"leader": "00574cam  2200181   4500", "fields": [{"001": "NEWS-126"}, {"008": "                                        "}, {"245": {"subfields": [{"a": "Radio-Tower Proposal Gets Hearing In Court"}], "ind1": " ", "ind2": " "}}, {"260": {"subfields": [{"c": "1989"}], "ind1": " ", "ind2": " "}}, {"650": {"subfields": [{"a": "Hoker Broadcasting Co."}], "ind1": " ", "ind2": " "}}, {"650": {"subfields": [{"a": "Wofx-Fm, Radio Tower"}], "ind1": " ", "ind2": " "}}, {"650": {"subfields": [{"a": "College Hill"}], "ind1": " ", "ind2": " "}}, {"773": {"subfields": [{"a": "Cincinnati Enquirer"}, {"g": "12/18/1989 A:10"}], "ind1": " ", "ind2": " "}}, {"907": {"subfields": [{"a": ".b10030098"}, {"b": "10-03-12"}, {"c": "08-31-12"}], "ind1": " ", "ind2": " "}}, {"998": {"subfields": [{"a": "2ma"}, {"b": "07-23-04"}, {"c": "m"}, {"d": "9"}, {"e": "-"}, {"f": "eng"}, {"h": "0"}], "ind1": " ", "ind2": " "}}, {"944": {"subfields": [{"a": "7"}], "ind1": " ", "ind2": " "}}, {"941": {"subfields": [{"a": "

In [11]:
record.decode_marc()

TypeError: Record.decode_marc() missing 1 required positional argument: 'marc'

In [12]:
# marc file stats
# https://gist.github.com/anarchivist/262826

import pymarc 
import sys

stats = {}

def tally(r):
    for f in r.fields:
        stats[f.tag] = stats.get(f.tag, 0) + 1

records = 0
with open('full-export.out', 'rb') as fh:
    reader = MARCReader(fh)
    for r in reader:
        records+=1
        tally(r)
# for filename in sys.argv[1:]:
#     for r in pymarc.MARCReader(''):
#         records += 1
#         tally(r)

#pymarc.map_xml(get_stats, *sys.argv[1:])

for e in stats:
    print("%s : %s" % (e, stats[e]))

print("%s total records" % records)

Unable to parse character 0x20 in g0=51 g1=69
Unable to parse character 0x20 in g0=51 g1=69
Unable to parse character 0x20 in g0=51 g1=69
Unable to parse character 0x20 in g0=51 g1=69
Unable to parse character 0x20 in g0=51 g1=69
Unable to parse character 0x20 in g0=51 g1=69
Unable to parse character 0x20 in g0=51 g1=69
Unable to parse character 0x20 in g0=51 g1=69
Unable to parse character 0x20 in g0=51 g1=69
Unable to parse character 0x20 in g0=51 g1=69
Unable to parse character 0x20 in g0=51 g1=69
Unable to parse character 0x20 in g0=51 g1=69
Unable to parse character 0x20 in g0=51 g1=69
Unable to parse character 0x20 in g0=51 g1=69
Unable to parse character 0x20 in g0=51 g1=69
Unable to parse character 0x20 in g0=51 g1=69
Unable to parse character 0x20 in g0=51 g1=69
Unable to parse character 0x20 in g0=51 g1=69
Unable to parse character 0x20 in g0=51 g1=69
Unable to parse character 0x20 in g0=51 g1=69
Unable to parse character 0x20 in g0=51 g1=69
Unable to parse character 0x20 in 

001 : 1577163
008 : 1886384
245 : 1886102
260 : 1885153
773 : 1886625
907 : 1886384
998 : 1886384
944 : 1780372
941 : 1780372
959 : 1577155
650 : 3873316
948 : 34446
100 : 7
600 : 9
540 : 15
250 : 5
658 : 1
775 : 28
007 : 2
733 : 3
700 : 1
730 : 5
490 : 1
770 : 3
655 : 1
1886384 total records


In [15]:
for f in record.fields:
    print(f)

=001  NEWS-242
=008  \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
=245  \\$aGift Shop Nucleus For Cincinnati-Russia Connection
=260  \\$c1989
=650  \\$aRussia Connection
=650  \\$aSister Cities Association Of Greater Cincinnati
=773  \\$aPost$g12/19/1989 N6:1
=907  \\$a.b10031042$b10-03-12$c08-31-12
=998  \\$a2ma$b07-23-04$cm$d9$e-$feng$h0
=944  \\$a102
=941  \\$a20040723$b0$cBATCH$d0
=959  \\$aNewspaper Article$mMAIN$tMICROFILM$lMAGSTACKS$xMICROFORM$rN$sY$p$0.00


In [78]:
[field.subfields for field in record.fields if field.tag == '907']

[['a', '.b10030098', 'b', '10-03-12', 'c', '08-31-12']]

In [92]:
print(
    'http://newsdex.chpl.org/search?/.b1003009/.b1003009/1%2C1%2C1%2CB/marc~b1003009 \n'
    f"bib_record_num: {record['907']['a']} \n",
    f"last_update: {record['907']['b']} \n",
    f"create_date: {record['907']['c']} \n",
    sep=''
)
    

http://newsdex.chpl.org/search?/.b1003009/.b1003009/1%2C1%2C1%2CB/marc~b1003009 
bib_record_num: .b10030098 
last_update: 10-03-12 
create_date: 08-31-12 



In [ ]:
# marc2csv example
# https://gist.github.com/mmccollow/348178
